# WiDS Datathon

https://www.kaggle.com/c/widsdatathon2020

1. Load Data
2. Basic Data Transformation
3. Drop columns with too little data. 
4. Identify data types of fields.
5. Fill missing data.
6. One Hot Encoding for categorical variables.
7. Pipeline (alternative to steps 5+6, with XGBoost).

In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', -1)

## Load Data

Load training data and data dictionary.

In [2]:
# import csv for training data set
training2 = pd.read_csv('training_v2.csv')

# import csv for 'WiDS Datathon 2020 Dictionary'
data_dictionary = pd.read_csv('WiDS Datathon 2020 Dictionary.csv')


## Initial Data Transformation

- Set 'encounter_id' as index.
- Drop 'patient_id'.
- Change 'gender' field data to binary rather than categorical.
- Drop columns with less than 70% complete data.

In [3]:
# Set 'encounter_id' as new index.
training2.set_index('encounter_id', inplace=True)

# Drop 'patient_id', since it is a unique identifier and not relevant for analysis.
training2.drop('patient_id', axis=1, inplace=True)

# Separate target from features
target = training2['hospital_death']
features = training2.drop('hospital_death', axis=1)

## Handle Missing Data: Drop Columns

We drop columns that have more than 50% missing data.

For later improvements, we may want to consider whether a field is populated or not as it's own variable.

In [4]:
# Create dataframe of count of non-NA values for each variable.
count_non_na = pd.DataFrame(features.count())

# Rename column.
count_non_na.rename(columns={0:"count"}, inplace=True)

# Add new column that calculates the percent of total rows that are populated by non-NA values.
total = features.shape[0]
count_non_na['pct_complete'] = count_non_na / total

# Drop columns with more than 50% missing data
columns_to_drop = count_non_na[count_non_na['pct_complete']<.50].index.values
features = features.drop(columns_to_drop, axis=1)


## Identify Field Data Types

In [5]:
data_dictionary.head()

,Category,Variable Name,Unit of Measure,Data Type,Description,Example
0,identifier,encounter_id,None,integer,Unique identifier associated with a patient unit stay,None
1,identifier,hospital_id,None,integer,Unique identifier associated with a hospital,None
2,identifier,patient_id,None,integer,Unique identifier associated with a patient,None
3,demographic,hospital_death,None,binary,Whether the patient died during this hospitalization,0
4,demographic,age,Years,numeric,The age of the patient on unit admission,None


In [6]:
# Update data dictionary to only include data that was not dropped.
data_type = data_dictionary[['Variable Name', 'Data Type']].set_index('Variable Name')
data_type = data_type.loc[features.columns,:]

In [7]:
data_type['Data Type'].unique()

array(['integer', 'numeric', 'string', 'binary'], dtype=object)

In [8]:
# Reclassify some incorrectly assigned data types from categorical to numeric.

data_type.loc[['bmi','apache_2_diagnosis','apache_3j_diagnosis'],'Data Type'] = 'numeric'


**Save variable data types as a csv**

We may use this in following notebookes. 

In [9]:
# Save data types df as csv

data_type.to_csv("variable_data_types.csv")

In [10]:
# Create lists for each data type

numeric_variables = list(data_type[data_type['Data Type']=='numeric'].index.values)
binary_variables = list(data_type[data_type['Data Type']=='binary'].index.values)
categorical_variables = list(data_type[data_type['Data Type']=='string'].index.values)
integer_variables = list(data_type[data_type['Data Type']=='integer'].index.values)

In [11]:
integer_variables

['hospital_id',
 'icu_id',
 'gcs_eyes_apache',
 'gcs_motor_apache',
 'gcs_verbal_apache']

In [12]:
features['gcs_eyes_apache'].unique()

array([ 3.,  1.,  4., nan,  2.])

For a first pass, we will drop the integer variables.

In [13]:
# Drop integer variables.
features.drop(integer_variables, axis=1, inplace=True)

## Manual vs Pipeline

If we are not interested in feature selection, we can bundle the rest of our data transformation with the fitting of a model, using a Pipeline. We take this approach at the end of the notebook.

Otherwise, we peform the next two steps in data transformation - fill missing data and one-hot encoding - manually to product a final dataset on which we can do feature selection.

In [14]:
# save copy of features data frame before we continue manual changes.
features_pipeline = features.copy()

## Feature Engineering

From Aiko's notebook.

In [17]:
#tally the total number of additional conditions
conditions = ['aids', 'cirrhosis', 'diabetes_mellitus', 'hepatic_failure',
       'immunosuppression', 'leukemia', 'lymphoma',
       'solid_tumor_with_metastasis']
features['tot_conditions'] = features[conditions].sum(axis=1)

#create a binary older than 60 category
features['over_60']= np.where((features['age']<60) , 1, 0)
features.head()

#create a binary icu type to group all the cardiac ones together
cardiac_icu_list = ['Cardiac ICU','CCU-CTICU','CSICU','CTICU']
features['cardiac_icu'] = np.where(features['icu_type'].isin(cardiac_icu_list),1,0)

#create a binary flag to indicate patient was likely surgical
features['surgery_likely'] = np.where(np.logical_or(features['elective_surgery']==1,features['apache_post_operative']==1),1,0)

features.head()

,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bun_apache,creatinine_apache,gcs_unable_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,tot_conditions,over_60,cardiac_icu,surgery_likely
encounter_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
66154,68.0,22.73,0,Caucasian,M,180.3,Floor,Floor,admit,CTICU,0.541667,0,73.9,113.0,502.01,0,0.0,31.0,2.51,0.0,168.0,118.0,27.4,0.0,40.0,36.0,134.0,39.3,0.0,14.1,68.0,37.0,68.0,37.0,119.0,72.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,131.0,73.0,131.0,73.0,39.9,37.2,68.0,63.0,68.0,63.0,119.0,108.0,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,131.0,115.0,131.0,115.0,39.5,37.5,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,0.10,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,1.0,0,1,0
114252,77.0,27.42,0,Caucasian,F,160.0,Floor,Floor,admit,Med-Surg ICU,0.927778,0,70.2,108.0,203.01,0,0.0,9.0,0.56,0.0,145.0,120.0,36.9,0.0,46.0,33.0,145.0,35.1,1.0,12.7,95.0,31.0,95.0,31.0,118.0,72.0,120.0,38.0,120.0,38.0,32.0,12.0,100.0,70.0,159.0,67.0,159.0,67.0,36.3,35.1,61.0,48.0,61.0,48.0,114.0,100.0,85.0,57.0,85.0,57.0,31.0,28.0,95.0,70.0,95.0,71.0,95.0,71.0,36.3,36.3,11.0,9.0,8.6,8.0,0.71,0.56,145.0,128.0,27.0,26.0,11.3,11.1,36.9,36.1,557.0,487.0,4.2,3.8,145.0,145.0,23.3,12.7,0.47,0.29,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,1.0,0,0,0
119783,25.0,31.95,0,Caucasian,F,172.7,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,0.000694,0,95.3,122.0,703.03,0,0.0,NaN,NaN,0.0,NaN,102.0,NaN,0.0,68.0,37.0,NaN,36.7,0.0,NaN,88.0,48.0,88.0,48.0,96.0,68.0,102.0,68.0,102.0,68.0,21.0,8.0,98.0,91.0,148.0,105.0,148.0,105.0,37.0,36.7,88.0,58.0,88.0,58.0,96.0,78.0,91.0,83.0,91.0,83.0,20.0,16.0,98.0,91.0,148.0,124.0,148.0,124.0,36.7,36.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,0.0,1,0,0
79267,81.0,22.64,1,Caucasian,F,165.1,Operating Room,Operating Room / Recovery,admit,CTICU,0.000694,0,61.7,203.0,1206.03,1,0.0,NaN,NaN,0.0,185.0,114.0,25.9,1.0,60.0,4.0,NaN,34.8,1.0,8.0,48.0,42.0,48.0,42.0,116.0,92.0,84.0,84.0,84.0,84.0,23.0,7.0,100.0,95.0,158.0,84.0,158.0,84.0,38.0,34.8,62.0,44.0,NaN,NaN,100.0,96.0,92.0,71.0,NaN,NaN,12.0,11.0,100.0,99.0,136.0,106.0,NaN,NaN,35.6,34.8,NaN,NaN,NaN,NaN,NaN,NaN,185.0,88.0,NaN,NaN,11.6,8.9,34.0,25.9,198.0,43.0,5.0,3.5,NaN,NaN,9.0,8.0,0.04,0.03,0.0,0.0,0.0,0.0,

## Fill Missing Data

If using Pipeline, skip this step.

We fill missing numeric data with the median of values in that field. We fill missing binary and categorical data with the mode.

In [18]:
# Missing numeric data - fill with median.

for variable in numeric_variables:
    median = features[variable].median()
    features[variable].fillna(median, inplace=True)

In [19]:
# Missing categorical and binary data - fill with mode.

for variable in categorical_variables:
    mode = features[variable].mode()[0]
    features[variable].fillna(mode, inplace=True)

for variable in binary_variables:
    mode = features[variable].mode()[0]
    features[variable].fillna(mode, inplace=True)

## Transform Categorical Columns with One Hot Encoding

In [20]:
#rename the 'Undefined diagnoses' to 'Undefined Diagnoses' in the APACHE 2 body system column
#otherwise these will be treated as two distinct values when generating dummy data which I assume is in error
features.replace(to_replace = 'Undefined diagnoses', value = 'Undefined Diagnoses',inplace=True)


In [21]:
features[categorical_variables].head(3)

,ethnicity,gender,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,apache_3j_bodysystem,apache_2_bodysystem
encounter_id,,,,,,,,
66154,Caucasian,M,Floor,Floor,admit,CTICU,Sepsis,Cardiovascular
114252,Caucasian,F,Floor,Floor,admit,Med-Surg ICU,Respiratory,Respiratory
119783,Caucasian,F,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,Metabolic,Metabolic


In [22]:
# Create dictionary of prefixes we will assign to dummy variables for each variable.
category_prefixes = {'ethnicity':'eth', 
                     'gender':'gender',
                     'hospital_admit_source':'hosp_admt', 
                     'icu_admit_source':'icu_admt',
                     'icu_stay_type':'icu_stay',
                     'icu_type':'icu_type',
                     'apache_3j_bodysystem':'a3',
                     'apache_2_bodysystem':'a2'
                    }

# Loop over categorical variables, converting each column to dummy variables 
#  then appending to data features data frame.
for variable in categorical_variables:
    var_series = features[variable].astype('category')
    var_pref = category_prefixes[variable]
    
    # Create dataframe of dummy variables
    var_df = pd.get_dummies(var_series, prefix=var_pref)
    
    # Concat the dummy variables to the features data set
    features = pd.concat([features, var_df], axis=1)

    
# Drop original categorical variable columns from dataset.
features = features.drop(categorical_variables, axis=1)

# Drop second gender column, since it is the same information stored in both. 
features.drop(['gender_M'], axis=1, inplace=True)

In [23]:
features.head(3)

,age,bmi,elective_surgery,height,pre_icu_los_days,readmission_status,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bun_apache,creatinine_apache,gcs_unable_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_heartrate_max,d1_heartrate_min,d1_mbp_max,d1_mbp_min,d1_mbp_noninvasive_max,d1_mbp_noninvasive_min,d1_resprate_max,d1_resprate_min,d1_spo2_max,d1_spo2_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,d1_temp_max,d1_temp_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,h1_mbp_max,h1_mbp_min,h1_mbp_noninvasive_max,h1_mbp_noninvasive_min,h1_resprate_max,h1_resprate_min,h1_spo2_max,h1_spo2_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_temp_max,h1_temp_min,d1_bun_max,d1_bun_min,d1_calcium_max,d1_calcium_min,d1_creatinine_max,d1_creatinine_min,d1_glucose_max,d1_glucose_min,d1_hco3_max,d1_hco3_min,d1_hemaglobin_max,d1_hemaglobin_min,d1_hematocrit_max,d1_hematocrit_min,d1_platelets_max,d1_platelets_min,d1_potassium_max,d1_potassium_min,d1_sodium_max,d1_sodium_min,d1_wbc_max,d1_wbc_min,apache_4a_hospital_death_prob,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,tot_conditions,over_60,cardiac_icu,surgery_likely,eth_African American,eth_Asian,eth_Caucasian,eth_Hispanic,eth_Native American,eth_Other/Unknown,gender_F,hosp_admt_Acute Care/Floor,hosp_admt_Chest Pain Center,hosp_admt_Direct Admit,hosp_admt_Emergency Department,hosp_admt_Floor,hosp_admt_ICU,hosp_admt_ICU to SDU,hosp_admt_Observation,hosp_admt_Operating Room,hosp_admt_Other,hosp_admt_Other Hospital,hosp_admt_Other ICU,hosp_admt_PACU,hosp_admt_Recovery Room,hosp_admt_Step-Down Unit (SDU),icu_admt_Accident & Emergency,icu_admt_Floor,icu_admt_Operating Room / Recovery,icu_admt_Other Hospital,icu_admt_Other ICU,icu_stay_admit,icu_stay_readmit,icu_stay_transfer,icu_type_CCU-CTICU,icu_type_CSICU,icu_type_CTICU,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU,a3_Cardiovascular,a3_Gastrointestinal,a3_Genitourinary,a3_Gynecological,a3_Hematological,a3_Metabolic,a3_Musculoskeletal/Skin,a3_Neurological,a3_Respiratory,a3_Sepsis,a3_Trauma,a2_Cardiovascular,a2_Gastrointestinal,a2_Haematologic,a2_Metabolic,a2_Neurologic,a2_Renal/Genitourinary,a2_Respiratory,a2_Trauma,a2_Undefined Diagnoses
encounter_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
66154,68.0,22.73,0,180.3,0.541667,0,73.9,113.0,502.01,0,0.0,31.0,2.51,0.0,168.0,118.0,27.4,0.0,40.0,36.0,134.0,39.3,0.0,14.1,68.0,37.0,68.0,37.0,119.0,72.0,89.0,46.0,89.0,46.0,34.0,10.0,100.0,74.0,131.0,73.0,131.0,73.0,39.9,37.2,68.0,63.0,68.0,63.0,119.0,108.0,86.0,85.0,86.0,85.0,26.0,18.0,100.0,74.0,131.0,115.0,131.0,115.0,39.5,37.5,31.0,30.0,8.5,7.4,2.51,2.23,168.0,109.0,19.0,15.0,8.9,8.9,27.4,27.4,233.0,233.0,4.0,3.4,136.0,134.0,14.1,14.1,0.10,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0
114252,77.0,27.42,0,160.0,0.927778,0,70.2,108.0,203.01,0,0.0,9.0,0.56,0.0,145.0,120.0,36.9,0.0,46.0,33.0,145.0,35.1,1.0,12.7,95.0,31.0,95.0,31.0,118.0,72.0,120.0,38.0,120.0,38.0,32.0,12.0,100.0,70.0,159.0,67.0,159.0,67.0,36.3,35.1,61.0,48.0,61.0,48.0,114.0,100.0,85.0,57.0,85.0,57.0,31.0,28.0,95.0,70.0,95.0,71.0,95.0,71.0,36.3,36.3,11.0,9.0,8.6,8.0,0.71,0.56,145.0,128.0,27.0,26.0,11.3,11.1,36.9,36.1,557.0,487.0,4.2,3.8,145.0,145.0,23.3,12.7,0.47,0.29,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,

**Save data as csv.**

We recombine the transformed features data with the target data so that we can store them in one file. The first step in following notebooks will be to load the data then to split the target from features again.

In [24]:
# Save transformed data

training_transformed = pd.concat([target, features], axis=1)
training_transformed.to_csv('training_transformed.csv')

## Train Model

In [34]:
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
#from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score#, balanced_accuracy_score
from sklearn.metrics import roc_auc_score


# Split test and train data set
X = features  #independent columns
y = target   #target column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)


# Fit model
my_model = XGBRegressor(n_estimators=500, n_jobs=2)
my_model.fit(X_train, y_train)


y_pred = my_model.predict(X_test)
print("ROC AUC Score new: " + str(roc_auc_score(y_test, y_pred)))

C:\Users\sowri\Miniconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[13:30:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
ROC AUC Score new: 0.8865681683519858


## Pipeline

- Fill Missing Data 
- One-Hot Encode Categorical Data

Following the tutorial here: https://www.kaggle.com/alexisbcook/pipelines

In [25]:
# Load saved pre-manual-processing dataset.
# Create test/train split.

#features = features_pipeline

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_pipeline, target, test_size=0.2, random_state=0, stratify=target)

In [26]:
### Define Preprocessing Step ###

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data -- we decided to use median for missing data.
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
# -- Replace missing data with mode.
# -- Use one hot encoding. 
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric_variables),
        ('cat', categorical_transformer, categorical_variables)
    ])

In [27]:
### Define the model ###

from xgboost import XGBRegressor

model = XGBRegressor(n_estimators=500, n_jobs=2)

In [28]:
### Create and evaluate pipeline ###

from sklearn.metrics import roc_auc_score

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_test)

# Evaluate the model
score = roc_auc_score(y_test, preds)
print('ROC AUC Score:', score)

C:\Users\sowri\Miniconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:32:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
ROC AUC Score: 0.8842713338469448


In [33]:
training2.nunique().sort_values()

readmission_status      1    
aids                    2    
immunosuppression       2    
diabetes_mellitus       2    
cirrhosis               2    
                       ..    
d1_pao2fio2ratio_min    4990 
d1_pao2fio2ratio_max    5194 
pre_icu_los_days        9757 
urineoutput_apache      24772
bmi                     34888
Length: 184, dtype: int64